In [105]:
#call packages
import spotipy
import spotipy.util as util
import pandas as pd
import sklearn
import numpy as np
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

## Build Dataset

In [2]:
#set token
token = util.prompt_for_user_token(username= "ComputerFace/PureBeing",
	client_id="643cfcf8dd054490b829709cdb6eec7c", 
	client_secret="3425490d0178401db9b9674d01afd068", 
	redirect_uri="https://github.com/ByronKKing/ComputerFace-PureBeing",
    scope='user-library-read')

spotify = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?response_type=code&client_id=643cfcf8dd054490b829709cdb6eec7c&redirect_uri=https%3A%2F%2Fgithub.com%2FByronKKing%2FComputerFace-PureBeing&scope=user-library-read in your browser


Enter the URL you were redirected to: https://github.com/ByronKKing/ComputerFace-PureBeing?code=AQAkUKTpRmng9Vg-Aw-Gdo5tyW44MSM-zB1LWxmh6ZAV2LHj5HTJ0kLwp3tN2ta6v0TsKjuS0g46xVkzIcvF1Qi9PAedWTev7pYJp20MAlPoJZmqeA5he54Mw40dHp_IuyAC4I5VIMDTD1HaRZmbrMHi0JLSkD04q5D5EEuv_mlJ-_skKR9Can9KijV-s2WyfcAFK2S8nT6G5CSVPiHXhoipZrIhLCDvdN-c8d4pMkaT_EGFOwCHXIR4bNT5




In [3]:
#get flylo artist
flylo_uri = 'spotify:artist:29XOeO6KIWxGthejQqn793'

In [4]:
#get album info
results = spotify.artist_albums(flylo_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

In [5]:
#save album ids in list
albumList = []
for album in albums:
    x = album['id']
    albumList.append(x)
    
#save album names in list
albumNames = []
for album in albums:
    x = album['name']
    albumNames.append(x)

In [6]:
#get information for all songs
songs = []
for i in range(0,len(albumList)):
    results = spotify.album_tracks(albumList[i])
    song = results['items']
    while results['next']:
        results = spotify.next(results)
        song.extend(results['items'])
    song[0]['album_name'] = albumNames[i]
    song[0]['album_id'] = albumList[i]
    songs.append(song)

In [8]:
#get song ids in list
songList = []
for i in range(1,len(songs)):
    x = songs[i]
    for song in x:
        y = song['id']
        songList.append(y)

In [9]:
#break into pieces
songs1 = songList[:int(round(len(songList)/2,0))]
songs2 = songList[int(round(len(songList)/2,0)):]

In [10]:
#retrieve song features, put into dataframe
results1 = spotify.audio_features(songs1)
results2 = spotify.audio_features(songs2)
results = results1 + results2
df = json_normalize(results)

In [11]:
#retrieve songs, put into dataframe
temp = []
for song in songs:
    temp.extend(song)
df1 = json_normalize(temp)

In [ ]:
#get audio analysis for each song
audioAnalysis = []
for song in songList:
    x = spotify.audio_analysis(song)
    x['song_id'] = song
    audioAnalysis.append(x)
df2 = json_normalize(audioAnalysis)

In [86]:
#get popularity for each song
tracks1 = spotify.tracks(songs1)
#tracks1['song_id'] = songs1
tracks2 = spotify.tracks(songs2)
#tracks2['song_id'] = songs2
tracks = {**tracks1, **tracks2}
df3 = json_normalize(tracks['tracks'])

In [45]:
#merge dataframes
songdf = pd.merge(df,df1,how='left',on='id')
songdf = pd.merge(songdf,df2,how='left',left_on='id',right_on='song_id')
songdf = pd.merge(songdf,df3[['id','popularity']],how='left',left_on='id',right_on='id')

## Clustering

In [109]:
#Convert DataFrame to matrix
mat = songdf[np.isfinite(songdf['popularity'])]._get_numeric_data().as_matrix()
#Using sklearn
km = KMeans(n_clusters=4).fit(mat)
#Get cluster assignment labels
labels = km.labels_
#Format results as a DataFrame
results = pd.DataFrame([songdf.index,labels]).T

In [ ]:
##cluster
##classifier
##get ep/single info
##see which eps are alike which albums--during similar period?